# GRASS GIS intro

In this first part we will demonstrate starting GRASS GIS, creating new project and basic data visualization.
We will first use GUI and then we replicate it in the notebook environment running on mybinder.com.

## GUI intro

## GRASS GIS in a notebook

### Notebook environment

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

However we will also show how to use GRASS from shell, for that we will use IPython magic:

In [ ]:
%%bash
pwd

### Start GRASS GIS

Create new empty location (project) that uses projection [UTM zone 17 N](https://epsg.io/6346).

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created location. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], universal_newlines=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("/home/anna/grassdata", "dix_park", "PERMANENT")

### Import data

Import prepared DSM, DTM and ortho. In this case no reprojection is done since the data CRS matches the location's CRS.

We also set computational region based on the imported raster (more on that later).

In [ ]:
%%bash
r.import input=dsm.tif output=dsm resample=bilinear
r.import input=ground.tif output=ground
r.import input=ortho.tif output=ortho

# set the region to match ortho raster
g.region raster=ortho

Download OSM data using Overpass Turbo.

We extract roads for our study area. The query should run upon running the cell below, so
it should be enough to *press Export - download as GeoJSON* and save it as `roads.geojson`.

In [ ]:
import IPython
IPython.display.IFrame('https://overpass-turbo.eu/s/1aGu', width=800, height=500)

Now we import the downloaded file. This time the data will be automatically reprojected because the CRS doesn't match.

In [ ]:
%%bash
v.import input=roads.geojson output=roads

Let's look at the available data in our location:

In [ ]:
%%bash
g.list type=raster,vector -m -t

### Data visualization

We will call d.rast/d.vect modules using new GRASS Jupyter API:

In [ ]:
# Start a GrassRenderer
img = gj.GrassRenderer()
# Add a raster and vector to the map
img.d_rast(map="ground")
img.d_vect(map="roads")
img.d_legend(raster="ground", flags='b')
# Display map
img.show()

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ortho")
img.d_vect(map="roads", width="2", color="yellow", where="name = 'Umstead Drive'")
img.show()

Here is how we can visualize data with folium:

In [ ]:
fig = gj.InteractiveMap(width=600)
# Add vector and layer control to map
fig.add_vector("roads")
fig.show()

TODO: 3D here

## GRASS GIS modules

GRASS functionality is available through modules (tools, functions). It has over 500 different modules in the core distribution and over 300 addon modules that can be used to prepare and analyze data.

Modules respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some modules have multiple dots in their names. For example, modules staring with v.net.* deal with vector network analysis and r.in.* modules import data into GRASS GIS spatial database.


### Finding and running a module

To find a module for your analysis, type the term into the **search box into the Modules tab** in the Layer Manager or just browse the module tree.
Most modules are also available from the **main menu**. For example, to find information about a raster map, use: *Raster → Reports and statistics → Basic raster metadata*.

If you already know the name of the module, you can just use it in the command line. The GUI offers a **Console tab with command line** specifically built for running GRASS GIS modules. If you type the module name there, you will get suggestions for automatic completion of the name. After pressing Enter, you will get GUI dialog for the module.

Last but not least, there is a module for finding modules:

In [ ]:
%%bash
g.search.modules keyword=zonal

In [ ]:
%%bash
r.univar --help

![title](img/picture.png)

### GUI  vs command line vs Python

GRASS modules can be executed either through a GUI, command line or Python interface. The GUI offers a user-friendly approach to executing modules where the user can navigate to data layers that they would like to analyze and modify processing options with simple check boxes.
The GUI also offers an easily accessible manual on how to execute a model. The command line interface allows users to execute a module using command prompts specific to that module. This is handy when you are running similar analyses with minor modification or are familiar with the module commands for quick efficient processing.

In this example we will show how the same module can be executed with GUI, in shell and in Python:


In [ ]:
%%bash
v.extract input=roads where="name = 'Umstead Drive'" output=umstead_drive_segmentsb

In [ ]:
gs.run_command("v.extract", input="roads", where="name = 'Umstead Drive'", output="umstead_drive_segments")

### Computational region

Computational region is an important raster concept in GRASS GIS.
Before we use a module to compute a new raster map, we must properly set the computational region. All raster computations will be performed in the specified extent and with the given resolution.

Among other things, this allows us to subset larger extent data for quicker testing of analysis, or to run an analysis of specific regions given by e.g. administrative units.

A few points to keep in mind:

 * computational region is defined by region extent and raster resolution
 * applies to all raster operations and vector to raster operations
 * persists between GRASS sessions, can be different for different mapsets
 * advantages: keeps your results consistent, avoid clipping, for computationally demanding tasks set region to smaller extent, check that your result is good and then set the computational region to the entire study area and rerun analysis
 * run `g.region -p` or in menu *Settings - Region - Display region* to see current region settings

In [ ]:
%%bash
g.region -p

The most common way to set region is **based on a raster map** - both extent and resolution. If the raster is added to Layer Manager, right click on it and select *Set computational region from selected map*. Or run the following g.region module (we include -p flag to always see the resulting region):

In [ ]:
%%bash
g.region -p raster=dsm

Computational region can be set also **using a vector map**. In that case, only extent is set (as vector map does not have any resolution - at least not in the way raster map does). In GUI, this can be done in the same way as for the raster map. In the command line, it looks like this:


In [ ]:
%%bash
g.region -p vector=roads | tail

However now the resolution was adjusted based on the extent of the vector map, it is no longer a nice rounded number. If that's not desired, we can set it explicitly using -a flag and parameter res. Now the resolution is aligned to even multiples of 2 (the units are the units of the current location, in our case meters):


In [ ]:
%%bash
g.region -p vector=roads res=2 -a

Often we need to set computational extent based on vector map, but take the resolution and alignment from a raster map:

In [ ]:
%%bash
g.region -p vector=roads align=dsm